# CobaltStrike Malleable Amazon browsing traffic profile
Detects Malleable Amazon Profile

## Rule Content
```
- title: CobaltStrike Malleable Amazon browsing traffic profile
  id: 953b895e-5cc9-454b-b183-7f3db555452e
  status: experimental
  description: Detects Malleable Amazon Profile
  references:
  - https://github.com/rsmudge/Malleable-C2-Profiles/blob/master/normal/amazon.profile
  - https://www.hybrid-analysis.com/sample/ee5eca8648e45e2fea9dac0d920ef1a1792d8690c41ee7f20343de1927cc88b9?environmentId=100
  author: Markus Neis
  tags:
  - attack.t1102
  logsource:
    category: proxy
    product: null
    service: null
  detection:
    selection1:
      c-useragent: Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like
        Gecko
      cs-method: GET
      c-uri: /s/ref=nb_sb_noss_1/167-3294888-0262949/field-keywords=books
      cs-host: www.amazon.com
      cs-cookie: '*=csm-hit=s-24KU11BB82RZSYGJ3BDK|1419899012996'
    selection2:
      c-useragent: Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like
        Gecko
      cs-method: POST
      c-uri: /N4215/adj/amzn.us.sr.aps
      cs-host: www.amazon.com
    condition: selection1 or selection2
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(c-useragent:"Mozilla\/5.0\ \(Windows\ NT\ 6.1;\ WOW64;\ Trident\/7.0;\ rv\:11.0\)\ like\ Gecko" AND cs-host:"www.amazon.com" AND ((cs-method:"GET" AND c-uri:"\/s\/ref\=nb_sb_noss_1\/167\-3294888\-0262949\/field\-keywords\=books" AND cs-cookie.keyword:*\=csm\-hit\=s\-24KU11BB82RZSYGJ3BDK|1419899012996) OR (cs-method:"POST" AND c-uri:"\/N4215\/adj\/amzn.us.sr.aps")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()